In [1]:
import torch
import numpy as np
from datasets.openml_datasets import OpenML_Dataset

from utils.weakener import Weakener
from models.model import MLP
from utils.losses import PartialLoss,LBLoss,EMLoss,OSLCELoss,OSLBrierLoss
from utils.trainig_testing import train_model,evaluate_model,train_and_evaluate


In [2]:
Data = OpenML_Dataset('iris')
train_x, train_y, test_x, test_y = Data.get_data()

In [3]:
Weak = Weakener(Data.num_classes)
Weak.generate_M(model_class='pll')

In [4]:
Weak.M,Weak.Z

(array([[0.   , 0.375, 0.375],
        [0.375, 0.   , 0.375],
        [0.375, 0.375, 0.   ],
        [0.25 , 0.25 , 0.25 ]]),
 array([[0, 1, 1],
        [1, 0, 1],
        [1, 1, 0],
        [1, 1, 1]]))

In [25]:
z,w = Weak.generate_weak(train_y)

In [26]:
train_y[:5,:],z[:5],w[:5,:]

(tensor([[0., 0., 1.],
         [0., 1., 0.],
         [0., 0., 1.],
         [0., 1., 0.],
         [1., 0., 0.]]),
 tensor([3, 0, 0, 2, 2], dtype=torch.int32),
 tensor([[1., 1., 1.],
         [0., 1., 1.],
         [0., 1., 1.],
         [1., 1., 0.],
         [1., 1., 0.]], dtype=torch.float64))

In [27]:
Weak.generate_wl_priors()

array([0.29220765, 0.21428552, 0.24350683, 0.25      ])

In [28]:
mlp = MLP(Data.num_features,[],Data.num_classes)

In [29]:
logsoftmax = torch.nn.LogSoftmax(dim=1)
softmax = torch.nn.Softmax(dim=1)

logit  = mlp(train_x)
logp = logsoftmax(logit)
p = softmax(logit)
print(logp[:5],p[:5],torch.exp(logp)[:5])


tensor([[-4.1027, -1.6893, -0.2246],
        [-3.2781, -1.5813, -0.2789],
        [-4.3520, -2.5918, -0.0919],
        [-4.0323, -2.1490, -0.1443],
        [-1.7862, -1.0807, -0.7071]], grad_fn=<SliceBackward0>) tensor([[0.0165, 0.1847, 0.7988],
        [0.0377, 0.2057, 0.7566],
        [0.0129, 0.0749, 0.9122],
        [0.0177, 0.1166, 0.8657],
        [0.1676, 0.3394, 0.4931]], grad_fn=<SliceBackward0>) tensor([[0.0165, 0.1847, 0.7988],
        [0.0377, 0.2057, 0.7566],
        [0.0129, 0.0749, 0.9122],
        [0.0177, 0.1166, 0.8657],
        [0.1676, 0.3394, 0.4931]], grad_fn=<SliceBackward0>)


In [31]:
print(Weak.M,'\n\n',Weak.z[:5],'\n\n',Weak.M[Weak.z][:5,:])

[[0.    0.375 0.375]
 [0.375 0.    0.375]
 [0.375 0.375 0.   ]
 [0.25  0.25  0.25 ]] 

 tensor([3, 0, 0, 2, 2], dtype=torch.int32) 

 [[0.25  0.25  0.25 ]
 [0.    0.375 0.375]
 [0.    0.375 0.375]
 [0.375 0.375 0.   ]
 [0.375 0.375 0.   ]]


In [38]:
Q = p[:5,:] * torch.tensor(Weak.M[Weak.z][:5,:])
Q /= torch.sum(Q,dim=1,keepdim=True)

In [41]:
Q

tensor([[0.0165, 0.1847, 0.7988],
        [0.0000, 0.2138, 0.7862],
        [0.0000, 0.0759, 0.9241],
        [0.1320, 0.8680, 0.0000],
        [0.3306, 0.6694, 0.0000]], dtype=torch.float64, grad_fn=<DivBackward0>)

In [ ]:
class EMLoss(torch.nn.module):
    def __init__(self,M):
        super(EMLoss, self).__init__()
        self.logsoftmax = torch.nn.LogSoftmax(dim=1)
        self.M = M
    def forward(self,out,z):
        logp = logsoftmax(out)
        p = torch.exp(logp)
        Q = p * torch.tensor(self.M[z])
        Q /= torch.sum(Q,dim=1,keepdim=True)
        L = -torch.sum(Q*logp)
        return L





